# Various resampled periods from 9 months of ADR Mean

In [ ]:
import numpy as np
import warnings
import pandas as pd
import pyperclip as clip
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
csv_paths = ['data/BTCUSDT_1m_3m_back_from_2023-03-14.csv', 'data/BTCUSDT_1m_3m_back_from_2022-11-15.csv', 'data/BTCUSDT_1m_3m_back_from_2022-07-19.csv']
df = pd.DataFrame()
for csv_path in csv_paths:
    df = pd.concat([df, pd.read_csv(csv_path)])

df['date'] = pd.to_datetime(df['timestamp']).dt.date 
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)
df


In [ ]:
from typing import Tuple

def adr(df: pd.DataFrame) -> Tuple[float, float]:
    daily_high_low = df.groupby('date')['high', 'low'].agg(['max', 'min'])
    daily_high_low['adr'] = daily_high_low[('high', 'max')] - daily_high_low[('low', 'min')]
    average_adr = np.mean(daily_high_low['adr'])
    range_size = average_adr * 0.1
    return average_adr, range_size

can try: W, 2W, 3W, M

In [ ]:
def plot_adr_range_size_for_period_group(df: pd.DataFrame, resample_arg: str = 'M') -> str:
    # resample the dataframe into monthly periods
    groups = df.resample(resample_arg)
    groups
    # loop through each group and save to a new dataframe
    out = ''
    dict_list = []
    for name, group in groups:
        average_adr, range_size = adr(group)
        out += f"* name: {name},\n* average_adr: {average_adr},\n* range_size: {range_size}\n\n"
        dict_list.append({'date': name, 'average_adr': average_adr, 'range_size': range_size})

    df_range_sizes = pd.DataFrame(dict_list)
    df_range_sizes['date'] = pd.to_datetime(df_range_sizes['date'])
    df_range_sizes.set_index('date', inplace=True)
    # print(df_range_sizes)
    df_range_sizes.plot(title=f'Average ADR and Range Size for {name} Period Group ({resample_arg})')
    return out

## Get range for specific sample period

In [ ]:
out = plot_adr_range_size_for_period_group(df, 'M')

In [ ]:
clip.copy(out)

## Compare various sample periods

In [ ]:
periods = ['W', '2W', '3W', 'M', '2M', '3M']
for period in periods:
    plot_adr_range_size_for_period_group(df, period)

## Conclusions

There is enough variation to consider adjusting the range weekly based, on the pervious weeks average rather than relying on a 3 month average as suggested by the book